# Text Data Preprocessing

**Step 1: Connect to Google Drive**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


**Step 2: Import Libraries**

In [ ]:
import nltk
nltk.download('stopwords')
import numpy as np
import pandas as pd
from gensim.models.keyedvectors import KeyedVectors
import gc
from smart_open import open
from nltk.corpus import stopwords
from sklearn.metrics import classification_report
from matplotlib import pyplot as plt
from keras.layers import Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Flatten, LSTM, Dense, Attention, TimeDistributed, Sequential, Dropout
from keras.layers import LSTM, TimeDistributed
from keras.layers import Dot, Activation, Multiply, Permute, RepeatVector, Lambda, Softmax
from keras.layers import Layer, Reshape
from keras import backend as K

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


**Step 3: Loading Data**

In [ ]:
dataset1 = np.array(pd.read_csv('/content/drive/My Drive/dev_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
dataset2 = np.array(pd.read_csv('/content/drive/My Drive/full_test_split.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
dataset3 = np.array(pd.read_csv('/content/drive/My Drive/train_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]

dataset = np.concatenate((dataset1, np.concatenate((dataset2, dataset3))))
countPos = 0

def checkPosNeg(dataset, index):
    for i in range(0, len(dataset)):
        if(dataset[i][0] == index):
            return dataset[i][1]
    return 0

Data = []
Y = []

countPos = 0
index = -1
Data_test = []
Y_test = []

In [ ]:
text_train_ids = [int(item[0]) for item in dataset3]
text_dev_ids = [int(item[0]) for item in dataset1]
text_test_ids = [int(item[0]) for item in dataset2]

**Step 4: Obtaining Corresponding Transcript**

In [ ]:

for i in range(0, len(dataset3)):
    val = checkPosNeg(dataset, dataset3[i][0])
    Y.append(val)
    try:
        fileName = "/content/drive/My Drive/" + str(int(dataset3[i][0])) + "_TRANSCRIPT.csv"
        Data.append(np.array(pd.read_csv(fileName,delimiter='\t',encoding='utf-8', engine='python'))[:, 2:4])
    except Exception as e:
        print(e)

In [ ]:
for i in range(0, len(dataset1)):
    val = checkPosNeg(dataset, dataset1[i][0])
    Y.append(val)
    try:
        fileName = "/content/drive/My Drive/" + str(int(dataset1[i][0])) + "_TRANSCRIPT.csv"
        Data.append(np.array(pd.read_csv(fileName, delimiter='\t', encoding='utf-8', engine='python'))[:, 2:4])
    except Exception as e:
        print(e)

In [ ]:

for i in range(0, len(dataset2)):
    Y_test.append(checkPosNeg(dataset, dataset2[i][0]))
    try:
        fileName = "/content/drive/My Drive/" + str(int(dataset2[i][0])) + "_TRANSCRIPT.csv"
        Data_test.append(np.array(pd.read_csv(fileName,delimiter='\t',encoding='utf-8', engine='python'))[:, 2:4])
    except Exception as e:
        print(e)


Y = np.array(Y)
Data2 = []

Data2_test = []
Y_test = np.array(Y_test)

for i in range(0, len(Data)):
    script = []
    for k in range(1, len(Data[i])):
        if(Data[i][k][0] == "Participant"):
            script.append(Data[i][k][1])
    Data2.append(script)

for i in range(0, len(Data_test)):
    script = []
    for k in range(1, len(Data_test[i])):
        if(Data_test[i][k][0] == "Participant"):
            script.append(Data_test[i][k][1])
    Data2_test.append(script)

Data = []
Data_test = []
gc.collect()
Data2 = np.array(Data2)
Data2_test = np.array(Data2_test)

<ipython-input-10-9c111b65c987>:33: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Data2 = np.array(Data2)
<ipython-input-10-9c111b65c987>:34: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  Data2_test = np.array(Data2_test)


**Step 5: Text Preprocessing using word2vec**

In [ ]:
model = KeyedVectors.load_word2vec_format('/content/drive/My Drive/GoogleNews-vectors-negative300.bin', binary=True)
stop_words = set(stopwords.words('english'))


def Thresholding(Y_pred, threshold):
  Y_pred2 = []
  for i in range(len(Y_pred)):
    if(Y_pred[i] < threshold):
      Y_pred2.append(0)
    else:
      Y_pred2.append(1)

  return np.array(Y_pred2)

def remove_StopWords(sentence):
    filtered_sentence = []
    for w in sentence:
        if w not in stop_words:
            filtered_sentence.append(w)

    return filtered_sentence

def checkAcc(Y_pred, Y_test):
    correct = 0
    for i in range(len(Y_pred)):
        if(Y_pred[i] == Y_test[i]):
            correct+=1

    return float(correct)/len(Y_pred)

def upsample(X_train,Y_train):
    # Separate majority and minority classes
    X_train_0 = X_train[Y_train==0]
    X_train_1 = X_train[Y_train==1]

    # Compute the difference in size
    size = X_train_0.shape[0] - X_train_1.shape[0]

    # If the minority class is smaller, upsample it
    if size > 0:
      # Randomly select 'size' samples from the minority class with replacement
      indices = np.random.choice(X_train_1.shape[0], size=size, replace=True)
      X_upsampled = X_train_1[indices]
      Y_upsampled = np.ones(size)

      # Concatenate the upsampled minority class and the original majority class
      X_train = np.concatenate((X_train_0, X_train_1, X_upsampled))
      Y_train = np.concatenate((np.zeros(X_train_0.shape[0]), np.ones(X_train_1.shape[0]), Y_upsampled))

    return np.array(X_train), np.array(Y_train)


max_num_words = 20
max_num_sentence = 250

#train_data
finalMatrix = np.zeros((Data2.shape[0], max_num_sentence, max_num_words, 300))
max_length_sent = 0
sent = ""
for k in range(Data2.shape[0]):
    if(max_length_sent < len(Data2[k])):
      max_length_sent = len(Data2[k])
      sent = Data2[k]
    for i in range(min(max_num_sentence, len(Data2[k]))):
    	try:
    	  sentence = Data2[k][i].split(" ")
    	except:
    	  continue
    	sentence = remove_StopWords(sentence)
    	for j in range(min(max_num_words, len(sentence))):
    		try:
    		  word = sentence[j]
    		  # print("Before", word)
    		  if(word[0] == '<'):
    		    if(word.find('>')!=-1):
    		      word = word[1:-1]
    		    else:
    		      word = word[1:]
    		  else:
    		    if(word.find('>')!=-1):
    		      word = word[0:-1]
    		  finalMatrix[k][i][j] = np.array(model[word])
    		except Exception as e:
    			continue

#Test_data
max_length_sent = 0
finalMatrix_test = np.zeros((Data2_test.shape[0], max_num_sentence, max_num_words, 300))
# print(finalMatrix_test.shape)
for k in range(Data2_test.shape[0]):
    if(max_length_sent < len(Data2_test[k])):
      max_length_sent = len(Data2_test[k])
      sent = Data2_test[k]
    for i in range(min(max_num_sentence, len(Data2_test[k]))):
    	try:
    	  sentence = Data2_test[k][i].split(" ")
    	except:
    	  continue
    	sentence = remove_StopWords(sentence)
    	for j in range(min(max_num_words, len(sentence))):
    		try:
    		  word = sentence[j]
    		  if(word[0] == '<'):
    		    if(word.find('>')!=-1):
    		      word = word[1:-1]
    		    else:
    		      word = word[1:]
    		  else:
    		    if(word.find('>')!=-1):
    		      word = word[0:-1]
    		  finalMatrix_test[k][i][j] = np.array(model[word])
    		except Exception as e:
    			continue

Data2 = []
Data2_test = []
model = []
stop_words = []
gc.collect()
print(finalMatrix.shape)
print(Y.shape)

(142, 250, 20, 300)
(142,)


In [ ]:
print(finalMatrix.shape)
print(Y.shape)

(142, 250, 20, 300)
(142,)


In [ ]:
finalMatrix_ids = [int(item[0]) for item in finalMatrix]
finalMatrix_ids

In [ ]:
print(Y_test.shape)

(47,)


**Step 6: Class Balancing**

In [ ]:
finalMatrix, Y = upsample(finalMatrix,Y)
finalMatrix_test, Y_test = upsample(finalMatrix_test,Y_test)

**Step 7: Save Preprocessed Data**

In [ ]:
np.save(file="/content/drive/My Drive/finalMatrix_1.npy", arr=finalMatrix)
np.save(file="/content/drive/My Drive/text_Y_1.npy", arr=Y)
np.save(file="/content/drive/My Drive/finalMatrix_test.npy", arr=finalMatrix_test)
np.save(file="/content/drive/My Drive/text_Y_test.npy", arr=Y_test)